# Loan Data Analysis
Yingcan Carol Wang

## Data Background
For this project, I used a dataset from Lending Club - a US online P2P lending company. Through Lending Club, investors provide funds for borrowers and earn a profit based on the risk they take (depending on the borrowers' credit score). Lending Club makes money by collecting fees from both borrowers and investors. This dataset mostly includes information from the borrower's side, ranging from the year 2007 to 2015.  

## Project Goals
1. Cluster borrowers to different types so as to facilitate targetted marketing and risk management.
<br><br>
2. Forcast and predict overall loan amount to help estimate market demand and run campaigns targeting at borrowers or investors at the right time.
<br><br>
3. Predict bad loan so as to reduce risk for the platform and its investors.


## General Data Cleaning

In [1]:
# Import libraries 
import pandas as pd
import numpy as np
import tensorflow as tf

# for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# others
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
import collections
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import time
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('loan.csv', low_memory=False)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-2015,171.62,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-2014,649.91,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,10.0,0.0,5598.0,21.0,37.0,f,0.0,0.0,12226.302212,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-2015,357.48,NaN,Jan-2015,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-2016,67.79,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape

(887379, 74)

The dataset contains 74 columns, but not all of them would be useful for this project. I pick the following features that are most relevant to the three goals of the project:
* loan_amnt
* funded_amnt_inv (the total amount committed by investors for that loan at that point in time)
* term (the number of payments on the loan, either in 36 or 60 months)
* int_rate (interest rate)
* installment
* grade (loan grade based on credit score)
* emp_length (employment time)
* home_ownership
* annual_inc (annual income)
* verification_status (whether income is verified)
* issue_d (the month which the loan was funded)
* loan_status 
* purpose (borrowing purpose)
* addr_state (the US state provided by the borrower as the address)
* dti (debt-income ratio)
* delinq_2yrs (the number of delinquency in the past 2 years)
* inq_last_6mths (the number of inquiries in the past 6 months) 
* total_acc (the total number of current credit lines)
* out_prncp (remaining outstanding principal for total amount funded)
* pub_rec (the number of derogatory public records)
* revol_bal (total credit revolving balance)
* revol_util (the amount of credit the borrower is using relative to all available revolving credit)

In [40]:
df_selected = df[['loan_amnt','funded_amnt_inv','term','int_rate','installment','grade',
                  'emp_length','home_ownership','annual_inc','verification_status','issue_d',
                  'loan_status','purpose',
                  'addr_state','dti','delinq_2yrs','inq_last_6mths',
                  'total_acc',
                  'out_prncp','pub_rec','revol_bal','revol_util']]

In [41]:
# check missing data
df_selected.isnull().sum()

loan_amnt                  0
funded_amnt_inv            0
term                       0
int_rate                   0
installment                0
grade                      0
emp_length             44825
home_ownership             0
annual_inc                 4
verification_status        0
issue_d                    0
loan_status                0
purpose                    0
addr_state                 0
dti                        0
delinq_2yrs               29
inq_last_6mths            29
total_acc                 29
out_prncp                  0
pub_rec                   29
revol_bal                  0
revol_util               502
dtype: int64

Since the row number of missing data only takes up ~5% of all rows and there is enough amount of data (887379 rows), deleting them will not affect our statistical power. 

In [116]:
# delete missing data
df_cleaned = df_selected.dropna()
df_cleaned.shape

(842067, 22)

In [45]:
df_cleaned.head(3)

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,total_acc,out_prncp,pub_rec,revol_bal,revol_util
0,5000.0,4975.0,36 months,10.65,162.87,B,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,credit_card,AZ,27.65,0.0,1.0,9.0,0.0,0.0,13648.0,83.7
1,2500.0,2500.0,60 months,15.27,59.83,C,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,car,GA,1.00,0.0,5.0,4.0,0.0,0.0,1687.0,9.4
2,2400.0,2400.0,36 months,15.96,84.33,C,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,small_business,IL,8.72,0.0,2.0,10.0,0.0,0.0,2956.0,98.5


In [49]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 842067 entries, 0 to 887378
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   loan_amnt            842067 non-null  float64
 1   funded_amnt_inv      842067 non-null  float64
 2   term                 842067 non-null  object 
 3   int_rate             842067 non-null  float64
 4   installment          842067 non-null  float64
 5   grade                842067 non-null  object 
 6   emp_length           842067 non-null  object 
 7   home_ownership       842067 non-null  object 
 8   annual_inc           842067 non-null  float64
 9   verification_status  842067 non-null  object 
 10  issue_d              842067 non-null  object 
 11  loan_status          842067 non-null  object 
 12  purpose              842067 non-null  object 
 13  addr_state           842067 non-null  object 
 14  dti                  842067 non-null  float64
 15  delinq_2yrs      

In [50]:
df_cleaned.describe()

,loan_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,total_acc,out_prncp,pub_rec,revol_bal,revol_util
count,842067.000000,842067.000000,842067.000000,842067.000000,8.420670e+05,842067.000000,842067.000000,842067.000000,842067.000000,842067.000000,842067.000000,8.420670e+05,842067.000000
mean,14915.479321,14860.788718,13.244072,440.779140,7.634501e+04,18.061077,0.317167,0.696321,25.339776,8466.188997,0.189016,1.708633e+04,55.228812
std,8449.246664,8457.198501,4.385640,244.625379,6.563192e+04,8.277712,0.865776,1.000607,11.845235,8554.460606,0.576540,2.261416e+04,23.814760
min,500.000000,0.000000,5.320000,15.670000,1.896000e+03,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,8325.000000,8150.000000,9.990000,263.930000,4.700000e+04,11.880000,0.000000,0.000000,17.000000,0.000000,0.000000,6.533000e+03,37.900000
50%,13200.000000,13150.000000,12.990000,386.740000,6.500000e+04,17.580000,0.000000,0.000000,24.000000,6542.560000,0.000000,1.201100e+04,56.200000
75%,20000.000000,20000.000000,16.200000,577.500000,9.000000e+04,23.830000,0.000000,1.000000,32.000000,13805.760000,0.000000,2.102300e+04,73.700000
max,35000.000000,35000.000000,28.990000,1445.460000,9.500000e+06,380.530000,39.000000,33.000000,169.000000,49372.860000,86.000000,2.904836e+06,892.300000


Next, we need to correct some features' data types and combine some catogories in a few categorical fetures. 

In [74]:
df_cleaned['emp_length'].value_counts()

10+ years    291417
2 years       78831
< 1 year      70538
3 years       69991
1 year        57055
5 years       55677
4 years       52503
7 years       44566
8 years       43929
6 years       42926
9 years       34634
Name: emp_length, dtype: int64

In [117]:
#column 'emp_length' can be treated as continuous data, so convert it to integer
#'< 1 year' will be 0, '10+ years' will be 10
emp_length = df_cleaned['emp_length'].str.split(" ",expand=True)[0]
df_cleaned['employment_length'] = emp_length.replace(["10+","<"],[10,0])

In [118]:
# column issue_d should be converted to date type
df_cleaned['issue_date'] = pd.to_datetime(df_cleaned['issue_d'])

In [76]:
df_cleaned['loan_status'].value_counts()

Current                                                568447
Fully Paid                                             200203
Charged Off                                             42680
Late (31-120 days)                                      10863
Issued                                                   7826
In Grace Period                                          6013
Late (16-30 days)                                        2213
Does not meet the credit policy. Status:Fully Paid       1935
Default                                                  1147
Does not meet the credit policy. Status:Charged Off       740
Name: loan_status, dtype: int64

In [119]:
# All categories including Charged Off, Late, In Grace Period and Default 
# should be combine into a 'bad loan' category.
bad_loan_list = ['Charged Off','Late (31-120 days)','In Grace Period','Late (16-30 days)',
                 'Default','Does not meet the credit policy. Status:Charged Off']
df_cleaned['loan_status_combined'] = df_cleaned['loan_status'].replace(bad_loan_list,'bad_loan')

In [87]:
#convert delinq_2yrs, inq_last_6mths, total_acc and pub_rec to integer
df_cleaned['delinq_2yrs'] = df_cleaned['delinq_2yrs'].astype(int)
df_cleaned['inq_last_6mths'] = df_cleaned['inq_last_6mths'].astype(int)
df_cleaned['total_acc'] = df_cleaned['total_acc'].astype(int)
df_cleaned['pub_rec'] = df_cleaned['pub_rec'].astype(int)

## Borrower Clustering
In order to cluster borrowers into different types, we need to choose the most relevant features first. We can summarise these features into the following 4 categories:
* Demographic info
    * addr_state
* Consumption demand
    * loan_amnt
    * total_acc
    * purpose
    * revol_util
* Loan repayment ability
    * emp_length
    * home_ownership
    * annual_inc
    * dti
    * revol_bal (the lower the better)
* Loan repayment willingness
    * grade (A is the best)
    * delinq_2yrs (the less the better)
    * inq_last_6mths (the less the better?)
    * pub_rec (the less the better)

<br>With these 4 categories, we can cluster borrowers using 6 different combinations:
1. loan repayment ability and willingness
2. consumption demand and loan repayment ability
3. consumption demand and loan repayment willingness
4. consumption demand in different states
5. loan repayment ability in different states
6. loan repayment willingness in different states

<br>What I need to do next is to choose the most representative feature within each category. Within the consumption demand category, 'loan_amnt' and 'purpose' respectively reflect the quantity and quality of the demand for loans. As for loan repayment ability and willingness, a heatmap reflecting the correlation between these features and loan status would be most helpful for feature selection.

<br>It is also important to scale all the features to be used before clustering. 
